In [1]:
import urllib.request, json

with urllib.request.urlopen(
    "https://api.steinhq.com/v1/storages/5e736c1db88d3d04ae0815b3/Raw_Data"
) as url:
    data = json.loads(url.read().decode())
    

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(data)

In [4]:
df["Notes"][30:35]

30              Travelled to Thailand and Malaysia
31                             Travelled from Iran
32                             Travelled from Iran
33                             Travelled from Oman
34    Travel from Italy on 29/02/2020 through Doha
Name: Notes, dtype: object

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [16]:
# import spacy
# from spacy import displacy
# from spacy.matcher import Matcher

# nlp = spacy.load("en_core_web_sm")
# matcher = Matcher(nlp.vocab)
# matched_sents = []  # Collect data of matched sentences to be visualized

In [17]:
# pattern = [{"LEMMA": "travel"}, {"LOWER": "from"}, {"POS": "PROPN"}]
# matcher.add("TravelledFrom", None, pattern)
# doc = nlp("Travelled from Italy and China")
# matches = matcher(doc)
# matches

In [18]:
# for match_id, start, end in matches:
#     string_id = nlp.vocab.strings[match_id]  # Get string representation
#     span = doc[start:end]  # The matched span
#     print(span.text)

In [19]:
# from spacy.tokens import Token
# def find_travelled_from(sents):
#     matcher = Matcher(nlp.vocab)
#     pattern = [{"LEMMA": "travel"}, {"LOWER": "from"}]
#     matcher.add("TravelledFrom", None, pattern)
#     Token.set_extension("is_travelled_from", default=False, force=True)
#     for sent in sents:
#         doc = nlp(sent)
#         matches = matcher(doc)
#         travelled_from = []
#         for match_id, start, end in matches:
#             if doc.vocab.strings[match_id] == "TravelledFrom":
#                 travelled_from.append(doc[start:end])
#         with doc.retokenize() as retokenizer:
#             for span in travelled_from:
#                 retokenizer.merge(span)
#                 for token in span:
#                     token._.is_travelled_from = True
#         for token in doc:
#             if token._.is_travelled_from:
#                 print(token.text, token._.is_travelled_from)
        

In [20]:
# input_sents = list(df["Notes"][30:35])
# find_travelled_from(input_sents)

In [21]:
# df["Notes"][35]

In [22]:
# def find_travelled_from(sents):
#     matcher = Matcher(nlp.vocab)
#     pattern = [{"LEMMA": "travel"}, {}]
#     matcher.add("Travel", None, pattern)
#     for sent in sents:
#         doc = nlp(sent)
#         matches = matcher(doc)
#         for match_id, start, end in matches:
#             string_id = nlp.vocab.strings[match_id]  # Get string representation
#             span = doc[start:end]  # The matched span
#             print("Input: ", sent, "\nPlace: ", span.text.split()[-1])

In [23]:
from spacy.tokens import Span

def get_travel_status(span):
    if span.label_ =="GPE":
        prev_token = span.doc[span.start - 1]
        if prev_token.text in ("from", "through", "via", "Via"):
            return("from")
        elif prev_token.text in ("to", "and"):
            return("to")
        return "to"

# Register the Span extension as 'travel_status'
Span.set_extension("travel_status", getter=get_travel_status, force=True)

In [24]:
from spacy.tokens import Span

def get_nationality(span):
    if span.label_ =="NORP":
        return span.text

# Register the Span extension as 'nationality'
Span.set_extension("nationality", getter=get_nationality, force=True)

doc = nlp("Indian Tourist")
print([(ent.text, ent._.nationality) for ent in doc.ents])

[('Indian', 'Indian')]


In [25]:
import re
def find_travelled(sents):
    for sent in sents:
        s = re.sub(r'[^\w\s]',' ',sent)
        doc = nlp(s)
        print(sent)
        for ent in doc.ents:
            if ent._.travel_status:
                print(f"\tTravel {ent._.travel_status} {ent.text}")
            if ent._.nationality:
                print(f"\tNationality {ent._.nationality}")

In [26]:
find_travelled(list(df["Notes"][20:40]))

Italian tourist (wife of P6)
	Nationality Italian
Family members of P4
Family members of P4
Family members of P4
Family members of P4
Family members of P4
Family members of P4
Indian who accompanied the Italian tourists
	Nationality Indian
	Nationality Italian
Travelled from Italy
	Travel from Italy
Travelled to Iran
	Travel to Iran
Travelled to Thailand and Malaysia
	Travel to Thailand
	Travel to Malaysia
Travelled from Iran
	Travel from Iran
Travelled from Iran
	Travel from Iran
Travelled from Oman
	Travel from Oman
Travel from Italy on 29/02/2020 through Doha
	Travel from Italy
	Travel from Doha
Travel from Italy on 29/02/2020 through Doha
	Travel from Italy
	Travel from Doha
Travel from Italy on 29/02/2020 through Doha
	Travel from Italy
	Travel from Doha
Relative and neighbour of P35-P37
Relative  and neighbour of P35-P37
No travel history


In [ ]:
find_travelled(list(df["Notes"]))